# DATA MINING - CRICKET SCORE PREDICTION- INSPIRED FROM KAGGLE

using algorithm- Linear, Lass0 and Random Forest

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('T20.csv')

In [3]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63888 entries, 0 to 63887
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        63888 non-null  int64  
 1   match_id          63888 non-null  int64  
 2   batting_team      63888 non-null  object 
 3   bowling_team      63888 non-null  object 
 4   ball              63888 non-null  float64
 5   runs              63888 non-null  int64  
 6   player_dismissed  63888 non-null  object 
 7   city              55340 non-null  object 
 8   venue             63888 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.4+ MB


In [5]:
df['match_id'].unique()

array([  2,   3,   4,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  23,  24,  25,  26,  27,  31,  32,  33,  34,  35,  36,  37,
        50,  51,  52,  53,  54,  55,  56,  57,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  75,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 106, 107, 108, 120, 122, 125, 127,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 171, 172, 193, 194, 195,
       196, 197, 198, 219, 220, 221, 222, 223, 224, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 250, 251, 252, 268, 270, 271, 272, 273, 276, 279, 304, 305,
       306, 307, 308, 309, 310, 311, 312, 383, 384, 394, 395, 421, 422,
       423, 431, 432, 433, 434, 435, 436, 440, 441, 442, 443, 44

In [6]:
df[df['city'].isnull()]['venue'][0].split(' ')[0]

'Melbourne'

In [7]:
df['batting_team'].unique()

array(['Australia', 'Bangladesh', 'New Zealand', 'South Africa',
       'England', 'West Indies', 'India', 'Pakistan', 'Afghanistan',
       'Sri Lanka'], dtype=object)

In [8]:
cricket_ground = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x : x[0]), df['city'])

In [9]:
df['city'] = cricket_ground

In [10]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [11]:
df = df[df['city'].isin(eligible_cities)]

In [12]:
df['current_score'] = df.groupby('match_id').cumsum()['runs']

In [13]:
df['over'] = df['ball'].apply(lambda x : str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x : str(x).split(".")[1])

In [14]:
df['balls_bowled'] = (df['over'].astype('int')*6 + df['ball_no'].astype('int'))

In [15]:
df['balls_left'] = 120 - df['balls_bowled']

In [16]:
# removing outliers:
df['balls_left'] = df['balls_left'].apply(lambda x: 0 if x < 0 else x)

In [17]:
df['player_dismissed'].apply(lambda x: 1 if x != '0' else '0')

0        0
1        0
2        0
3        0
4        0
        ..
63883    0
63884    0
63885    1
63886    0
63887    0
Name: player_dismissed, Length: 50501, dtype: object

In [18]:
df['wicket_taken'] = df['player_dismissed'].apply(lambda x: 1 if x != '0' else 0)

In [19]:
df['wicket_taken'] = df['wicket_taken'].astype('int')

In [20]:
df['wicket_taken'] = df.groupby('match_id').cumsum()['wicket_taken']

In [21]:
df['wickets_left'] = 10 - df['wicket_taken']

In [22]:
df['current_runrate'] = (df['current_score']*6) / df['balls_bowled']

In [23]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

C:\Users\Deepali Thakur\AppData\Local\Temp\ipykernel_16960\3055083752.py:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city', 'player_dismissed', 'venue'], dtype='object')
  last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())
C:\Users\Deepali Thakur\AppData\Local\Temp\ipykernel_16960\3055083752.py:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city', 'venue'], dtype='object')
  last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())


In [24]:
df['last_five'] = last_five

In [25]:
df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [26]:
df= df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','current_runrate','last_five','runs_x']]

In [27]:
df.dropna(inplace=True)

In [28]:
df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,current_runrate,last_five,runs_x
29,Australia,Sri Lanka,Melbourne,43,90,10,8.600000,43.0,168
30,Australia,Sri Lanka,Melbourne,44,89,10,8.516129,44.0,168
31,Australia,Sri Lanka,Melbourne,45,88,10,8.437500,45.0,168
32,Australia,Sri Lanka,Melbourne,45,87,10,8.181818,44.0,168
33,Australia,Sri Lanka,Melbourne,45,86,10,7.941176,42.0,168
...,...,...,...,...,...,...,...,...,...
50496,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128
50497,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
50498,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
50499,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128


In [29]:
df = df.sample(df.shape[0])

In [30]:
# --- Data Preprocessing ---
# Converting categorical features using OneHotEncoding method
encoded_df = pd.get_dummies(data=df, columns=['batting_team', 'bowling_team','city'])

In [31]:
encoded_df.head(10)

,current_score,balls_left,wickets_left,current_runrate,last_five,runs_x,batting_team_Afghanistan,batting_team_Australia,batting_team_Bangladesh,batting_team_England,...,city_Mumbai,city_Nagpur,city_Nottingham,city_Pallekele,city_Southampton,city_St Kitts,city_St Lucia,city_Sydney,city_Trinidad,city_Wellington
33494,36,88,8,6.750000,32.0,152,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12377,109,47,8,8.958904,47.0,190,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19994,133,22,8,8.142857,43.0,180,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
25567,142,7,4,7.539823,41.0,148,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
44488,75,76,9,10.227273,55.0,204,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1921,66,61,7,6.711864,31.0,137,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
48129,48,75,8,6.400000,33.0,146,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1067,113,42,8,8.692308,46.0,202,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50154,145,4,6,7.500000,33.0,150,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6073,107,33,9,7.379310,38.0,176,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df.corr()

,current_score,balls_left,wickets_left,current_runrate,last_five,runs_x
current_score,1.000000,-0.862662,-0.385264,0.541206,0.528279,0.426976
balls_left,-0.862662,1.000000,0.666707,-0.083889,-0.194463,-0.035072
wickets_left,-0.385264,0.666707,1.000000,0.302932,0.236179,0.387297
current_runrate,0.541206,-0.083889,0.302932,1.000000,0.768312,0.757212
last_five,0.528279,-0.194463,0.236179,0.768312,1.000000,0.614438
runs_x,0.426976,-0.035072,0.387297,0.757212,0.614438,1.000000


In [33]:
X = encoded_df.drop(columns=['runs_x'])
y = encoded_df['runs_x']

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [35]:
# Standardinzing the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [36]:
X_train = scaler.fit_transform(X_train)

In [37]:
X_test = scaler.transform(X_test)

In [38]:
# scaler.inverse_transform(X-train), if required to be back in original form.

In [39]:
# Hyperparameter tuning
from sklearn.model_selection import cross_val_score

In [40]:
# --- Model Building ---
# Linear Regression Model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression()

In [41]:
mse= cross_val_score(regressor,X_train,y_train,scoring = 'neg_mean_squared_error',cv= 10)

In [42]:
np.mean(mse)

-316.1534180443492

In [43]:
y_pred_lr = regressor.predict(X_test)

In [44]:
y_pred_lr

array([192.63329919, 250.20751794, 147.95800622, ..., 167.51708825,
       145.73876794, 168.51251062])

In [45]:
import seaborn as sns
#sns.displot(y_pred_lr-y_test,kind='kde')

In [46]:
from sklearn.metrics import mean_absolute_error as mae , mean_squared_error as mse , accuracy_score
print("---- Linear Regression - Model Evaluation ----")
print("Mean Absolute Error (MAE): {}".format(mae(y_test, y_pred_lr)))
print("Mean Squared Error (MSE): {}".format(mse(y_test, y_pred_lr)))
print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(y_test, y_pred_lr))))
print('\nR-squared:', regressor.score(X_train, y_train))

---- Linear Regression - Model Evaluation ----
Mean Absolute Error (MAE): 13.035866049644603
Mean Squared Error (MSE): 308.67473169599225
Root Mean Squared Error (RMSE): 17.56914146155105

R-squared: 0.6932197739669635


Lasso Regression Application

In [47]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [48]:
#lasso=Lasso()

#parameters={'alpha':[1e-8,1e-3,1e-2,1,5,10,20],}

#lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=10)

#lasso_regressor.fit(X_train,y_train)
#print(lasso_regressor.best_params_)
#print(lasso_regressor.best_score_)

In [49]:
#y_pred_ls = lasso_regressor.predict(X_test)

In [50]:
#y_pred_ls

In [51]:

#sns.displot(y_pred_ls - y_test,kind='kde')

In [52]:
#print("---- Lasso - Model Evaluation ----")
#print("Mean Absolute Error (MAE): {}".format(mae(y_test, y_pred_ls)))
#print("Mean Squared Error (MSE): {}".format(mse(y_test, y_pred_ls)))
#print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(y_test, y_pred_ls))))
#print('\nR-squared:', lasso_regressor.score(X_train, y_train))

Random Forest Algo

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
clf = RandomForestRegressor()
parameters1 = {'n_estimators':[75,100],'max_depth':[11,21,31]}
#(n_estimators = 100,max_depth=21, random_state=0)
clf_regressor = GridSearchCV(clf,parameters1,scoring='neg_mean_squared_error',cv=10)
clf_regressor.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [11, 21, 31], 'n_estimators': [75, 100]},
             scoring='neg_mean_squared_error')

In [54]:
print(clf_regressor.best_params_)
print(clf_regressor.best_score_)

{'max_depth': 31, 'n_estimators': 75}
-31.458345839067942


In [55]:
y_pred_clf = clf_regressor.predict(X_test)

In [56]:
y_pred_clf

array([145.49333333, 241.        , 124.72975968, ..., 136.66666667,
       145.53671081, 192.04481838])

In [57]:
import seaborn as sns
#sns.displot(y_pred_clf - y_test,kind='kde')

In [59]:
print("---- RandomForest Regression - Model Evaluation ----")
print("Mean Absolute Error (MAE): {}".format(mae(y_test, y_pred_clf)))
print("Mean Squared Error (MSE): {}".format(mse(y_test, y_pred_clf)))
print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(y_test, y_pred_clf))))
print('\nR-squared:', clf_regressor.score(X_train, y_train))

---- RandomForest Regression - Model Evaluation ----
Mean Absolute Error (MAE): 2.407182593663626
Mean Squared Error (MSE): 23.856918087796878
Root Mean Squared Error (RMSE): 4.884354418733031

R-squared: -4.988365056352677


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
rf = RandomForestRegressor(n_estimators =100, max_depth= 31)

# Calculate the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    rf, X_train, y_train, cv=10, scoring='neg_mean_squared_error', 
    train_sizes=np.linspace(0.1, 1.0, 10))


In [ ]:
# Plot the learning curve
train_scores_mean = -np.mean(train_scores, axis=1)
test_scores_mean = -np.mean(test_scores, axis=1)
plt.plot(train_sizes, train_scores_mean, 'o-', color='r',
         label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g',
         label='Cross-validation score')
plt.xlabel('Training set size')
plt.ylabel('Mean squared error')
#plt.title('Random forest regression learning curve')
plt.legend(loc='best')
plt.show()

# XGB Booster

In [ ]:
from xgboost import XGBRegressor
#from numpy import absolute
#from sklearn.model_selection import RepeatedKFold
#xgb = XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1)
# define model evaluation method
#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
#scores = cross_val_score(xgb, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
#scores = absolute(scores)

#print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )


# define model
xgb = XGBRegressor()

# Define the parameter grid to search
param_grid = {'learning_rate': [0.1,0.2],
              'max_depth': [10, 12],
              'n_estimators': [500,1000]}

# Define the GridSearchCV object
grid = GridSearchCV(xgb, param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid.fit(X_train, y_train)

# Print the best hyperparameters
print(grid.best_params_)


In [ ]:
y_pred_xgb = grid.predict(X_test)

In [ ]:
y_pred_xgb

In [ ]:
import seaborn as sns
sns.displot(y_pred_xgb - y_test,kind='kde')

In [ ]:
print("---- XGB - Model Evaluation ----")
print("Mean Absolute Error (MAE): {}".format(mae(y_test, y_pred_xgb)))
print("Mean Squared Error (MSE): {}".format(mse(y_test, y_pred_xgb)))
print("Root Mean Squared Error (RMSE): {}".format(np.sqrt(mse(y_test, y_pred_xgb))))
print('\nR-squared:', grid.score(X_train, y_train))

In [ ]:
# define the model
#model = XGBRegressor()
# define the datasets to evaluate each iteration
#evalset = [(X_train, y_train), (X_test,y_test)]
# fit the model
#model.fit(X_train, y_train, eval_metric='logloss', eval_set=evalset)
# evaluate performance
#yhat = model.predict(X_test)

#from matplotlib import pyplot
# retrieve performance metrics
#results = model.evals_result()
# plot learning curves
#pyplot.plot(results['validation_0']['logloss'], label='train')
#pyplot.plot(results['validation_1']['logloss'], label='test')
# show the legend
#pyplot.legend()
# show the plot
#pyplot.show()

In [ ]:
#from sklearn.linear_model import Ridge
#from sklearn.model_selection import GridSearchCV
#ridge=Ridge()
#parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40]}
#ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
#ridge_regressor.fit(X_train,y_train)
#print(ridge_regressor.best_params_)
#print(ridge_regressor.best_score_)
#y_pred_ridge = ridge_regressor.predict(X_test)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import xgboost as xgb

# Define the XGBRegressor model with chosen hyperparameters
xgb_model = xgb.XGBRegressor(learning_rate=0.1, max_depth=3, n_estimators=100)

# Define the range of training set sizes to use
train_sizes = np.linspace(0.1, 1.0, 10)

# Generate the learning curve data
train_sizes, train_scores, validation_scores = learning_curve(
    xgb_model, X_train, y_train, train_sizes=train_sizes, cv=5)

# Calculate the mean and standard deviation of the training and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

# Plot the learning curve
plt.plot(train_sizes, train_scores_mean, label='Training score')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2)
plt.plot(train_sizes, validation_scores_mean, label='Validation score')
plt.fill_between(train_sizes, validation_scores_mean - validation_scores_std,
                 validation_scores_mean + validation_scores_std, alpha=0.2)
plt.xlabel('Number of training examples')
plt.ylabel('R-squared score')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import xgboost as xgb

xgb_model = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=1000)


# Calculate the learning curve
train_sizes, train_scores, test_scores = learning_curve(
    xgb_model, X_train, y_train, cv=10, scoring='neg_mean_squared_error', 
    train_sizes=np.linspace(0.1, 1.0, 10))


In [ ]:
# Plot the learning curve
train_scores_mean = -np.mean(train_scores, axis=1)
test_scores_mean = -np.mean(test_scores, axis=1)
plt.plot(train_sizes, train_scores_mean, 'o-', color='r',
         label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g',
         label='Cross-validation score')
plt.xlabel('Training set size')
plt.ylabel('Mean squared error')
#plt.title('Random forest regression learning curve')
plt.legend(loc='best')
plt.show()